# NEURON tutorial - part 1

## Sections and segments

NEURON is a simulator that is designed to simulate neurons with realistic extended morphologies. How can NEURON do this? Basically in NEURON, a neuron is represented as a collection of cylinders, called **sections**.

![](images/r18_1_sections.png)

Here we display a pyramidal neuron with two different sections are highlited (red). The problem is that neurons oftentimes have sections with quite different sizes such as long and thick (left) and short and tiny (right). To deal with this problem, each section is divided into **segments**, and so we can divide big sections with many segments while we can keep small ones with less or no segmenting.

Therefore, we have four different morphological parameters for each section,

1. `L`: length of a section,
2. `diam`: diameter,
3. `nseg`: a number of segments,
4. `x`: a *relative* coordinate along a section. `x=0` and `x=1` represent one and the other end, and `x=0.5` represents a middle point.

![Example section](./images/cylinder.png)


Then, every dynamical variable such as membrane potential is computed segment by segment. For example, if our section has `nseg=3`, membrane potential at three different points, v(x=0.0), v(x=0.5), and v(x=1.0), can be different from each other. However, if `nseg=1`, v(x) will be exactly the same no matter which point we measure --- the section is isopotential. Having this in mind, let's open NEURON and create some sections.

In [2]:
from neuron import h, gui

axon = h.Section(name="axon")

Now let's inspect what's in `axon`.

In [3]:
h.psection(axon)

soma { nseg=1  L=100  Ra=35.4
	/*location 0 attached to cell 0*/
	/* First segment only */
	insert morphology { diam=500}
	insert capacitance { cm=1}
}


1.0

We can see that `axon` is created with default values for the shape parameters and also others such as `cm` and `Ra`, which represent *capacitance* and *axial resistance* of this section, respectively. `L=100` means that the length is 100 μm, and `diam=500` means that the diameter is 500 μm. 

### Default units used in NEURON

|Quantity | Unit|
|:--------|:-----|
|Time | ms |
|Voltage | mV |
|Current density (distributed) | mA/cm2 |
|Current (point process) | nA |
|Concentration | mM |
|Specific capacitance | uF/cm2 |
|Length | um |
|Conductance density (distributed) | S/cm2 |
|Conductance (point process) | uS |
|Axial resistance | ohm cm |
|Resistance | Mohm|
